In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("covid_toy.csv")

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
28,16,Male,104.0,Mild,Kolkata
63,10,Male,100.0,Mild,Bangalore
97,20,Female,101.0,Mild,Bangalore
81,65,Male,99.0,Mild,Delhi
40,49,Female,102.0,Mild,Delhi
...,...,...,...,...,...
87,47,Male,101.0,Strong,Bangalore
54,60,Female,99.0,Mild,Mumbai
31,83,Male,103.0,Mild,Kolkata
93,27,Male,100.0,Mild,Kolkata


### without column transformer

In [8]:
# simple imputer on fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [9]:
X_train_fever.shape

(80, 1)

In [10]:
X_train_fever[:5]
# all null values are replaced by mean

array([[104.],
       [100.],
       [101.],
       [ 99.],
       [102.]])

In [11]:
# ordinal encoding on cough
df['cough'].unique()

array(['Mild', 'Strong'], dtype=object)

In [12]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [13]:
X_train_cough.shape

(80, 1)

In [14]:
X_train_cough[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [15]:
# onehotencoding on gender and city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_g_c = ohe.fit_transform(X_train[['gender','city']])
X_test_g_c = ohe.fit_transform(X_test[['gender','city']])

In [16]:
X_train_g_c.shape

(80, 4)

In [17]:
X_train_g_c[:5]

array([[1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [18]:
df['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [19]:
df['city'].unique()

array(['Kolkata', 'Delhi', 'Mumbai', 'Bangalore'], dtype=object)

In [20]:
# separating age from X_train and X_test
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [21]:
X_train_age.shape

(80, 1)

In [22]:
X_train_age[:5]

array([[16],
       [10],
       [20],
       [65],
       [49]], dtype=int64)

In [23]:
# cocatenating all transformed columns
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_g_c,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_g_c,X_test_cough),axis=1)

In [24]:
X_train_transformed.shape

(80, 7)

In [25]:
X_train_transformed[:5]

array([[ 16., 104.,   1.,   0.,   1.,   0.,   0.],
       [ 10., 100.,   1.,   0.,   0.,   0.,   0.],
       [ 20., 101.,   0.,   0.,   0.,   0.,   0.],
       [ 65.,  99.,   1.,   1.,   0.,   0.,   0.],
       [ 49., 102.,   0.,   1.,   0.,   0.,   0.]])

### with column transformer

In [ ]:
# passthrough means to keep the column as it is without doing any transformation

In [26]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse=False),['gender','city'])
],remainder='passthrough')

In [27]:
transformer.fit_transform(X_train)[:2]

array([[104.,   0.,   1.,   0.,   1.,   0.,  16.],
       [100.,   0.,   1.,   0.,   0.,   0.,  10.]])

In [28]:
transformer.fit_transform(X_train).shape

(80, 7)

In [29]:
transformer.transform(X_test)[:2]

array([[101.,   0.,   1.,   1.,   0.,   0.,  15.],
       [101.,   0.,   1.,   1.,   0.,   0.,  19.]])

In [30]:
transformer.transform(X_test).shape

(20, 7)